In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
from torchvision.models import resnet18
import pydicom

from sklearn.model_selection import train_test_split

In [6]:
# Training and testing data
train_input_data = ['Noisy Images/1-1_speckle_noise.png', 'Noisy Images/1-2_speckle_noise.png']
test_input_data = ['Noisy Images/1-6_speckle_noise.png']
train_output_data = ['Photos/1-1.dcm', 'Photos/1-2.dcm']
test_output_data = ['Photos/1-6.dcm']

In [7]:
# Load the data
def load_data(input_data, output_data):
    input_images = [plt.imread(image) for image in input_data]
    output_images = [pydicom.dcmread(image).pixel_array for image in output_data]
    return input_images, output_images

train_input_images, train_output_images = load_data(train_input_data, train_output_data)
test_input_images, test_output_images = load_data(test_input_data, test_output_data)

# Convert the data to tensors
def convert_to_tensor(images):
    return torch.tensor(images).float()

train_input_images = convert_to_tensor(train_input_images)
train_output_images = convert_to_tensor(train_output_images)
test_input_images = convert_to_tensor(test_input_images)
test_output_images = convert_to_tensor(test_output_images)

# Normalize the data
def normalize_data(images):
    return images / 255

train_input_images = normalize_data(train_input_images)
train_output_images = normalize_data(train_output_images)
test_input_images = normalize_data(test_input_images)
test_output_images = normalize_data(test_output_images)

# Load the model
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
model = DenoisingAutoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)

# Train the model
num_epochs = 100
outputs = []

for epoch in range(num_epochs):
    for data in train_input_images:
        data = data.unsqueeze(0).unsqueeze(0)
        recon = model(data)
        loss = criterion(recon, data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item()}')
    outputs.append((epoch, data, recon))

# Test the model
for data in test_input_images:
    data = data.unsqueeze(0).unsqueeze(0)
    recon = model(data)
    loss = criterion(recon, data)
    print(f'Test Loss:{loss.item()}')

# Plot the results
for k in range(0, num_epochs, 10):
    plt.figure(figsize=(9, 2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= 9:
            break
        plt.subplot(2, 9, i+1)
        item = item.squeeze()
        plt.imshow(item)
    for i, item in enumerate(recon):
        if i >= 9:
            break
        plt.subplot(2, 9, 9+i+1)
        item = item.squeeze()
        plt.imshow(item)
    plt.show()

# Save the model
torch.save(model.state_dict(), 'model.pth')

/var/folders/tj/t9m9g2r12fd6s4n9vlvhyddm0000gn/T/ipykernel_28287/3937646280.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  return torch.tensor(images).float()
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Sem 1/Deep Learning in Medicla Imaging/ImageResolutionEnhancement/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1, 1, 720, 960])) that is different to the input size (torch.Size([1, 1, 240, 320])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (320) must match the size of tensor b (960) at non-singleton dimension 3